In [45]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from collections import Counter

In [46]:
class MyKNNClf:
    def __init__(self, k=3, metric='euclidean', weight='uniform'):
        # Сохраняем параметр k внутри экземпляра
        self.k = k
        # Инициализация переменной train_size
        self.train_size = None
        # Инициализация переменных для хранения тренировочных данных
        self.X_train = None
        self.y_train = None
        self.metric = metric
        self.weight = weight

    def __str__(self):
        # Возвращаем строку в требуемом формате
        return f"MyKNNClf class: k={self.k}"
    
      # Метод обучения
    def fit(self, X: pd.DataFrame, y: pd.Series):
        # Сохраняем X и y внутри объекта класса
        self.X_train = X
        self.y_train = y
        # Записываем размер тренировочной выборки (количество строк, столбцов)
        self.train_size = X.shape  # shape возвращает кортеж (строки, столбцы)
        
    # Вычисление расстояния в зависимости от метрики
    def _compute_distance(self, row1, row2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((row1 - row2) ** 2))
        elif self.metric == 'chebyshev':
            return np.max(np.abs(row1 - row2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(row1 - row2))
        elif self.metric == 'cosine':
            dot_product = np.dot(row1, row2)
            norm1 = np.linalg.norm(row1)
            norm2 = np.linalg.norm(row2)
            return 1 - (dot_product / (norm1 * norm2))
        else:
            raise ValueError(f"Unknown metric: {self.metric}")
        
    # # Метод для вычисления Евклидова расстояния
    # def _euclidean_distance(self, row1, row2):
    #     return np.sqrt(np.sum((row1 - row2) ** 2))
    
     # Вычисление весов в зависимости от параметра weight
    def _compute_weights(self, distances):
        if self.weight == 'uniform':
            return np.ones(len(distances))  # Все веса одинаковые
        elif self.weight == 'rank':
            # Вес обратно пропорционален рангу (месту в сортированном списке)
            return 1 / (np.arange(1, len(distances) + 1))
        elif self.weight == 'distance':
            # Вес обратно пропорционален расстоянию (если расстояние равно 0, то вес задаем большим значением)
            distances = np.array(distances)
            with np.errstate(divide='ignore'):  # Игнорируем деление на 0
                weights = 1 / distances
                weights[distances == 0] = np.inf  # Если расстояние равно нулю, вес -> бесконечность
            return weights
        else:
            raise ValueError(f"Unknown weight: {self.weight}")

     # Метод для предсказания классов
    def predict(self, X: pd.DataFrame):
        predictions = []
        
        for _, test_row in X.iterrows():
            distances = []
            for _, train_row in self.X_train.iterrows():
                dist = self._compute_distance(test_row, train_row)
                distances.append(dist)
            
            # Находим индексы k ближайших соседей
            nearest_neighbors_indices = np.argsort(distances)[:self.k]
            nearest_neighbors_classes = self.y_train.iloc[nearest_neighbors_indices]
            nearest_distances = [distances[i] for i in nearest_neighbors_indices]

            # Если вес 'uniform', то используем обычную моду
            if self.weight == 'uniform':
                most_common_class = Counter(nearest_neighbors_classes).most_common(1)[0][0]
            else:
                # Иначе вычисляем веса
                weights = self._compute_weights(nearest_distances)
                weighted_class_counts = Counter()
                for weight, neighbor_class in zip(weights, nearest_neighbors_classes):
                    weighted_class_counts[neighbor_class] += weight
                
                # Выбираем класс с наибольшим суммарным весом
                most_common_class = weighted_class_counts.most_common(1)[0][0]

            predictions.append(most_common_class)
        
        return predictions

    # Метод для предсказания вероятностей
    def predict_proba(self, X: pd.DataFrame):
        probabilities = []
        
        for _, test_row in X.iterrows():
            distances = []
            for _, train_row in self.X_train.iterrows():
                dist = self._compute_distance(test_row, train_row)
                distances.append(dist)
            
            # Находим индексы k ближайших соседей
            nearest_neighbors_indices = np.argsort(distances)[:self.k]
            nearest_neighbors_classes = self.y_train.iloc[nearest_neighbors_indices]
            nearest_distances = [distances[i] for i in nearest_neighbors_indices]

            # Если вес 'uniform', вероятность для класса 1 это доля соседей класса 1
            if self.weight == 'uniform':
                class_1_count = np.sum(nearest_neighbors_classes == 1)
                probability_class_1 = class_1_count / self.k
            else:
                # Иначе вычисляем веса
                weights = self._compute_weights(nearest_distances)
                weighted_class_1_sum = sum(w for w, cls in zip(weights, nearest_neighbors_classes) if cls == 1)
                total_weight = sum(weights)
                probability_class_1 = weighted_class_1_sum / total_weight
            
            probabilities.append(probability_class_1)
        
        return probabilities

In [48]:
# Тестирование Взвешенный KNN
def weighted_knn(k_value, metric_type, weight_type):
    # Генерация данных для бинарной классификации
    X, y = make_classification(n_samples=100, n_features=5, n_informative=3, n_classes=2, random_state=42)
    
    # Преобразуем в pandas DataFrame и Series
    X_df = pd.DataFrame(X)
    y_series = pd.Series(y)
    
    # Создаем и обучаем модель с указанной метрикой и весами
    knn = MyKNNClf(k=k_value, metric=metric_type, weight=weight_type)
    knn.fit(X_df, y_series)
    
    # Выполняем предсказание и предсказание вероятностей
    predictions = knn.predict(X_df)
    probabilities = knn.predict_proba(X_df)
    
    # Возвращаем количество предсказаний и сумму вероятностей
    return len(predictions), sum(probabilities)

# Пример вызова теста
weight_type = 'rank'  # или 'distance'
predictions_count, probabilities_sum = weighted_knn(k_value=3, metric_type='euclidean', weight_type=weight_type)
print(f"Sample Output: ({predictions_count}, {probabilities_sum:.1f})")

Sample Output: (100, 49.7)


In [41]:
# Тестирование метрик
def test_metric(k_value, metric_type):
    # Генерация данных для бинарной классификации
    X, y = make_classification(n_samples=100, n_features=5, n_informative=3, n_classes=2, random_state=42)
    
    # Преобразуем в pandas DataFrame и Series
    X_df = pd.DataFrame(X)
    y_series = pd.Series(y)
    
    # Создаем и обучаем модель с указанной метрикой
    knn = MyKNNClf(k=k_value, metric=metric_type)
    knn.fit(X_df, y_series)
    
    # Выполняем предсказание и предсказание вероятностей
    predictions = knn.predict(X_df)
    probabilities = knn.predict_proba(X_df)
    
    # Возвращаем количество предсказаний и сумму вероятностей
    return len(predictions), sum(probabilities)

# Пример вызова теста
metric_type = 'euclidean'
predictions_count, probabilities_sum = test_metric(k_value=3, metric_type=metric_type)
print(f"Sample Output: ({predictions_count}, {probabilities_sum:.1f})")

Sample Output: (100, 49.3)


In [30]:
# Тестирование предсказания
def test_knn(k_value):
    # Генерация данных для бинарной классификации
    X, y = make_classification(n_samples=100, n_features=5, n_informative=3, n_classes=2, random_state=42)
    
    # Преобразуем в pandas DataFrame и Series
    X_df = pd.DataFrame(X)
    y_series = pd.Series(y)
    
    # Создаем и обучаем модель
    knn = MyKNNClf(k=k_value)
    knn.fit(X_df, y_series)
    
    # Выполняем предсказание и предсказание вероятностей
    predictions = knn.predict(X_df)
    probabilities = knn.predict_proba(X_df)
    
    # Возвращаем количество предсказаний и сумму вероятностей
    return len(predictions), sum(probabilities)

# Пример вызова теста
k_value = 3
predictions_count, probabilities_sum = test_knn(k_value)
print(f"Sample Output: ({predictions_count}, {probabilities_sum})")

Sample Output: (100, 49.33333333333332)


In [28]:
# Тестирование обучения
def test_knn_with_datasets():
    # Список параметров для тестов
    test_params = [
        {"n_samples": 50, "n_features": 5, "n_informative": 2},
        {"n_samples": 100, "n_features": 10, "n_informative": 5},
        {"n_samples": 200, "n_features": 20, "n_informative": 10}
    ]

    # Создаем объект KNN
    knn = MyKNNClf()

    # Проходим по каждому набору параметров
    for idx, params in enumerate(test_params):
        # Генерация данных
        X, y = make_classification(n_samples=params['n_samples'],
                                   n_features=params['n_features'],
                                   n_informative=params['n_informative'],
                                   random_state=42)

        # Преобразование в pandas DataFrame и Series
        X_df = pd.DataFrame(X)
        y_series = pd.Series(y)

        # Обучение модели
        knn.fit(X_df, y_series)

        # Печать результата для каждого набора данных
        print(f"Test {idx + 1}:")
        print(f"Train size: {knn.train_size}")
        print("-" * 40)

test_knn_with_datasets()

Test 1:
Train size: (50, 5)
----------------------------------------
Test 2:
Train size: (100, 10)
----------------------------------------
Test 3:
Train size: (200, 20)
----------------------------------------


In [29]:
# Пример использования вызова класса
knn1 = MyKNNClf()
knn2 = MyKNNClf(5)
knn3 = MyKNNClf(10)

# Проверка
print(knn1)
print(knn2)
print(knn3)

MyKNNClf class: k=3
MyKNNClf class: k=5
MyKNNClf class: k=10
